In [1]:
 pip install --upgrade scikit-learn

     |████████████████████████████████| 22.3MB 1.5MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [2]:
!pip install --upgrade category_encoders

     |████████████████████████████████| 81kB 3.8MB/s 


# Logistic Regression

이번에는 [**Cardiovascular Disease dataset**](https://www.kaggle.com/sulianova/cardiovascular-disease-dataset) 데이터셋을 사용해 봅시다. 데이터에 대한 설명을 캐글 페이지에서 잘 살펴보시고 과제를 진행해 주세요!

### 1) 데이터를 전처리 하겠습니다.
- 데이터를 불러오세요.(read_csv 사용)
- id 칼럼은 drop하세요.
- missing value를 확인하세요.
- 중복된 샘플이 있는지 확인하세요.
- **중복된 샘플을 제거하고 남은 샘플의 수를 과제 제출폼에 제출하세요.**

In [3]:
# 데이터 불러오기
import pandas as pd
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/cardio/cardio_train.csv', sep=';')
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [4]:
# id컬럼 drop
df.drop('id', axis = 1, inplace = True)
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [5]:
# missing value확인
df.isnull().sum()

age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
active         0
cardio         0
dtype: int64

In [6]:
# 중복된 샘플 확인하기
df[df.duplicated()==True]

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
10562,20495,1,165,70.0,120,80,1,1,0,0,1,0
21784,16793,1,165,68.0,120,80,1,1,0,0,1,0
38505,18988,1,164,65.0,120,80,1,1,0,0,1,0
40365,14552,1,158,64.0,120,80,1,1,0,0,1,0
42450,18353,1,169,67.0,120,80,1,1,0,0,1,0
44653,16937,2,170,70.0,120,80,1,1,0,0,0,0
45125,21280,1,165,65.0,120,80,1,1,0,0,1,0
45748,22077,1,175,69.0,120,80,1,1,0,0,1,1
45810,21230,1,164,62.0,120,80,1,1,0,0,1,0
48917,21945,1,165,60.0,120,80,1,1,0,0,1,0


In [7]:
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [8]:
# 중복된 샘플 제거하기
df[df.duplicated()==True].index

for i in df[df.duplicated()==True].index:
  df.drop(i, axis = 0,inplace=True)  

print('남은 Sample의 수:',len(df))

# df.drop_duplicates() 함수 사용하는것이 더 편리하다.

남은 Sample의 수: 69976


### 2) 다운로드 한 훈련 데이터를 훈련/검증/테스트 세트로 나누어 주세요.
- 타겟 특성을 확인하고 타겟 변수를 나누어 줍니다.
- 훈련과 테스트 데이터로 나누어 줍니다(train_test_split 사용, train/test = 80/20, random_state=2)
- 훈련데이터를 훈련과 검증데이터로 나누어 줍니다(train_test_split 사용, train/validation = 80/20, random_state=2)
- **검증 데이터(X_val)의 갯수를 과제 제출폼에 제출하세요.**

타겟 특성 확인

Age : 나이

Height : 높이

Weight : 무게

Gender :  성별

ap_hi : 수축기 혈압

ap_lo : 이완기 혈압

Cholesterol : 콜레스테롤

gluc : 포도당

Smoke : 흡연

alco: 알코올 섭취

active : 신체 활동

cardio :  심혈과 질환의 유무

In [9]:
df.columns

Index(['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol',
       'gluc', 'smoke', 'alco', 'active', 'cardio'],
      dtype='object')

In [10]:
# feature/target 선정

target = 'cardio'
feature = df.columns[:-1]


# train, test셋 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[feature],df[target], test_size = 0.2, random_state = 2)

# train, validation셋 분리
X2_train, X_val, y2_train, y_val,  = train_test_split(X_train, y_train, test_size = 0.2, random_state = 2)

# 분리된 데이터 점검
print('검증데이터의 개수는',len(X_val),'개')

검증데이터의 개수는 11196 개


### 3) 사이킷런의 LogisticRegression을 사용해 모델을 만들어 보세요.
- 모델을 만들기 전에 기준모델을 설정합니다.
- **LogisticRegression 모델을 학습하고(max_iter=1000) 검증세트 정확도를 과제 제출폼에 제출하세요.**


In [11]:
# 기준모델 선정
# 기준모델 완성하기
major = y_train.mode()[0]
y_pred = [major] * len(y_train)

In [12]:
# LogisticRegression 모델 불러오기
from sklearn.linear_model import LogisticRegression


# 모델 학습하기
logistic = LogisticRegression(max_iter=1000)
logistic.fit(X2_train, y2_train)

LogisticRegression(max_iter=1000)

In [13]:
# 검증셋에 모델 정확도 테스트 하기
print('검증세트 정확도:',logistic.score(X_val, y_val))

검증세트 정확도: 0.7073061807788495


### 4) 데이터에서 범주형 성질을 지닌 특성을 찾아 원핫인코딩 하세요.
- category_encoders를 사용해서 'gender' 특성만 원핫인코딩 하세요.
- LogisticRegression 모델을 다시 학습합니다.
- **검증세트 정확도를 과제 제출폼에 제출하세요.**

In [14]:
from category_encoders import OneHotEncoder
encoder = OneHotEncoder(use_cat_names=True,cols='gender')
X2_train_encoded =  encoder.fit_transform(X2_train)
X_val_encoded = encoder.transform(X_val)
X2_train_encoded.head(3)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,age,gender_1.0,gender_2.0,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
221,17591,1,0,157,70.0,160,100,3,1,0,0,0
59211,19677,0,1,174,80.0,150,80,1,1,0,0,1
26654,23504,1,0,158,76.0,140,90,3,3,0,0,1


In [15]:
#logistic = LogisticRegression(max_iter=1000)
logistic.fit(X2_train_encoded, y2_train)

LogisticRegression(max_iter=1000)

In [16]:
# 검증셋에 모델 정확도 테스트 하기
print('검증세트 정확도:',logistic.score(X_val_encoded , y_val))

검증세트 정확도: 0.6944444444444444


### 5) 학습한 모델을 검증세트에서 최적화 시킨 후, 테스트세트에서 딱 한 번 정확도를 확인해 보세요.
- **테스트 세트 정확도를 과제 제출폼에 제출하세요.(0.712 이상)**

In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
# 검정세트에서 최적화
# StandardScaler
scaler = StandardScaler()
# 검증데이터 분리 전 최초의 train 데이터
X_train_encoded = encoder.fit_transform(X_train)
X_train_scaled = scaler.fit_transform(X_train)

# 최초의 test 데이터
X_test_encoded = encoder.transform(X_test)
X_test_scaled = scaler.transform(X_test
                                 )
# 모델 학습 
logistic.fit(X_train_scaled, y_train)

y_pred = logistic.predict(X_test_scaled)
print(accuracy_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


0.7284224064018291


## 🔥 도전과제:

### 6) EDA를 추가적으로 수행하고 가능한 방법으로 모델을 튜닝해서 결과를 만들어 보세요. 최소한 다음 내용들이 포함이 되어야 합니다.
- 새로운 특성을 최소한 한 개 이상 만드세요.
- 특성들을 확인해서 아웃라이어라고 생각되는 데이터를 가진 샘플을 제거하세요.
- [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) 을 사용하세요. (여러가지 종류의 Scalar를 어떤 상황에 적용해야 하는지 알아보고 사용해 보세요.)
- 만들어진 모델 성능을 리포팅하고 해석합니다.
